In [1]:
!pip install langchain-text-splitters
!pip install sentence-transformers
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


# load Văn bản hợp nhất-03-VBHN-BGTVT

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
file_path = '/content/Văn bản hợp nhất-03-VBHN-BGTVT.docx.txt'
with open(file_path, 'r') as file:
            content = file.read()

# split content to chunk

In [56]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dangvantuan/vietnamese-embedding-LongContext')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
    length_function=lambda x: len(tokenizer(x)["input_ids"]),
    is_separator_regex=False,
)

In [57]:
chunks_ = text_splitter.create_documents([content])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

Token indices sequence length is longer than the specified maximum sequence length for this model (84346 > 32768). Running this sequence through the model will result in indexing errors


The text has been broken down in 120 chunks.


In [58]:
max([len(chunk.split()) for chunk in chunks]), max([len(tokenizer(chunk)["input_ids"]) for chunk in chunks])

(859, 988)

# convert chunk to vector
I use dangvantuan/vietnamese-embedding model for embedding model.

In [59]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding-LongContext',
                                      trust_remote_code = True).cuda() #('thenlper/gte-large').cuda()
embeddings = embedding_model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

tensor([[0.9605]])


In [60]:
import torch
import numpy as np

embeddings_ = []
batch_size = 16

for _id in range(0, len(chunks)//batch_size + int(len(chunks) % batch_size != 0)):
    start_id = _id * batch_size
    end_id = min(len(chunks), start_id + batch_size)
    print(start_id, end_id)
    batch_chunks = [chunk for chunk in chunks[start_id:end_id]]
    with torch.no_grad():
        embeddings = embedding_model.encode(batch_chunks)
    embeddings_.append(embeddings)

embeddings = torch.tensor(np.concatenate(embeddings_, 0))
embeddings.shape

0 16
16 32
32 48
48 64
64 80
80 96
96 112
112 120


torch.Size([120, 768])

# load LLM model
I use Qwen 2.5 7B Instruct

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# convert question to embedding

In [126]:
#question = 'xe máy vi phạm nồng độ cồn bị phạt bao nhiêu tiền?'
#question = 'ô tô vượt đèn đỏ bị phạt bao nhiêu tiền?'
#question = 'ô tô vi phạm nồng độ cồn bị phạt bao nhiêu tiền?'
#question = 'vi phạm nồng độ cồn bị phạt bao nhiêu tiền?'
question = 'khi đi xe máy, để quên giấy tờ xe và không có giấy tờ xe có bị phạt giống nhau không?'
query_embedding = embedding_model.encode([question])
query_embedding = torch.tensor(query_embedding)
query_embedding.shape

torch.Size([1, 768])

# calulate top 20 similar chunk by cosine

In [127]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
scores = cos(query_embedding.cuda(), embeddings.cuda())

In [128]:
indices = scores.topk(20).indices.cpu().long().numpy()
indices

array([35, 11, 58, 99, 49, 52, 29, 15, 37, 20, 32, 18, 34, 30, 67, 13,  6,
       48, 57, 98])

In [129]:
import numpy as np
chunks_ = np.array(chunks)
chunks_[indices]

array(['Điều 22. Xử phạt các hành vi vi phạm quy định về điều kiện của người điều khiển xe máy chuyên dùng\n1. Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với một trong các hành vi vi phạm sau đây:\na) Người điều khiển xe máy chuyên dùng không đúng độ tuổi hoặc tuổi không phù hợp với ngành nghề theo quy định;\nb) Người điều khiển xe máy chuyên dùng không mang theo bằng (hoặc chứng chỉ) điều khiển, chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ;\nc) Người điều khiển xe máy chuyên dùng không mang theo Giấy đăng ký xe;\nd) Người điều khiển xe máy chuyên dùng không mang theo Giấy chứng nhận an toàn kỹ thuật và bảo vệ môi trường (đối với loại xe có quy định phải kiểm định).\n2. Phạt tiền từ 600.000 đồng đến 1.000.000 đồng đối với người điều khiển xe máy chuyên dùng không có bằng (hoặc chứng chỉ) điều khiển, chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ.\nMục 5. VI PHẠM QUY ĐỊNH VỀ VẬN TẢI ĐƯỜNG BỘ\nĐiều 23. Xử phạt người điều khiển xe ô tô chở hành khách,

# create question prompt

In [130]:
Contexts = "\n".join(chunks_[indices].tolist())
print(Contexts)

Điều 22. Xử phạt các hành vi vi phạm quy định về điều kiện của người điều khiển xe máy chuyên dùng
1. Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Người điều khiển xe máy chuyên dùng không đúng độ tuổi hoặc tuổi không phù hợp với ngành nghề theo quy định;
b) Người điều khiển xe máy chuyên dùng không mang theo bằng (hoặc chứng chỉ) điều khiển, chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ;
c) Người điều khiển xe máy chuyên dùng không mang theo Giấy đăng ký xe;
d) Người điều khiển xe máy chuyên dùng không mang theo Giấy chứng nhận an toàn kỹ thuật và bảo vệ môi trường (đối với loại xe có quy định phải kiểm định).
2. Phạt tiền từ 600.000 đồng đến 1.000.000 đồng đối với người điều khiển xe máy chuyên dùng không có bằng (hoặc chứng chỉ) điều khiển, chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ.
Mục 5. VI PHẠM QUY ĐỊNH VỀ VẬN TẢI ĐƯỜNG BỘ
Điều 23. Xử phạt người điều khiển xe ô tô chở hành khách, ô tô chở người 

In [131]:
question_prompt = f'''Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{Contexts}

Now here is the question you need to answer.

Question: {question}'''

In [132]:
print(question_prompt)

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
Điều 22. Xử phạt các hành vi vi phạm quy định về điều kiện của người điều khiển xe máy chuyên dùng
1. Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Người điều khiển xe máy chuyên dùng không đúng độ tuổi hoặc tuổi không phù hợp với ngành nghề theo quy định;
b) Người điều khiển xe máy chuyên dùng không mang theo bằng (hoặc chứng chỉ) điều khiển, chứng chỉ bồi dưỡng kiến thức pháp luật về giao thông đường bộ;
c) Người điều khiển xe máy chuyên dùng không mang theo Giấy đăng ký xe;
d) Người điều khiển xe máy chuyên dùng không mang theo Giấy chứng nhận an toàn kỹ thuật và bảo vệ môi trường (đối với loại xe có quy định phải ki

# Ask Qwen2 7B
Sorry because I don't have gpu to run it :(

In [ ]:
device = "cuda" # the device to load the model onto

messages = [
    {"role": "system", "content": "You are a helpful assistant. Please answer question more detail base on context!"},
    {"role": "user", "content": question_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.26 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.92 GiB is free. Process 77212 has 12.83 GiB memory in use. Of the allocated memory 12.38 GiB is allocated by PyTorch, and 331.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Ask Qwen2.5 72B
Because I can't run it on Colab Free tier, I use Together AI to test Qwen2.5 72B

In [19]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 1.9 MB/s eta 0:00:00


In [133]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-72B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Dựa trên thông tin từ Điều 30 của Nghị định, việc để quên giấy tờ xe và không có giấy tờ xe sẽ bị xử phạt khác nhau:

1. Trường hợp để quên giấy tờ xe:
- Người điều khiển phương tiện sẽ bị phạt tiền từ 100.000 đồng đến 200.000 đồng (Điều 30, khoản 3).

2. Trường hợp không có giấy tờ xe:
- Nếu người điều khiển phương tiện không có giấy tờ xe (không đăng ký xe), mức phạt sẽ cao hơn, từ 1.000.000 đồng đến 2.000.000 đồng (Điều 30, khoản 5, điểm b).

Ngoài ra, nếu người điều khiển phương tiện không xuất trình được giấy tờ xe khi bị kiểm tra, sẽ bị lập biên bản vi phạm hành chính và có thể bị tạm giữ phương tiện. Nếu sau đó xuất trình được giấy tờ, chỉ bị xử phạt về hành vi không mang theo giấy tờ (khoản 3 Điều 30).

Vì vậy, việc để quên giấy tờ xe và không có giấy tờ xe không bị phạt giống nhau, mà mức phạt cho trường hợp không có giấy tờ xe cao hơn nhiều.


In [134]:
import json

model = "Qwen2.5-72B-Instruct-Turbo"

with open(f"{question}_{model}.txt", 'w', encoding='utf-8') as f:
    f.write(f"#question_prompt:\n\n\n {question_prompt} \n\n\n#answer:\n\n\n {response.choices[0].message.content}")

# Ask Llama 3.1 405B
Because I can't run it on Colab Free tier, I use Together AI to test Llama 3.1 405B

In [135]:
import os
from google.colab import userdata
from together import Together

client = Together(api_key=userdata.get('togetherai_api'))

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    messages=[{"role": "user", "content": question_prompt}],
)
print(response.choices[0].message.content)

Không giống nhau. Trường hợp không mang giấy tờ xe sẽ bị phạt từ 100.000 đồng đến 200.000 đồng. Trường hợp không có giấy tờ xe sẽ bị phạt từ 800.000 đồng đến 1.000.000 đồng.


In [136]:
import json

model = "Meta-Llama-3.1-405B-Instruct-Turbo"

with open(f"{question}_{model}.txt", 'w', encoding='utf-8') as f:
    f.write(f"#question_prompt:\n\n\n {question_prompt} \n\n\n#answer:\n\n\n {response.choices[0].message.content}")

# The result base on LLM model and embedding model.

## This pipeline will better than short context. LLM can give correct answer for some questions

## The relevant chunk don't show enough contexts than LLm can't answer correctly!

## Chunks only split by token len, they don't have enough context about Article or Clause. Than the context maybe unclear for LLM.

## We need to rerank relevant chunk after get top 20 similar chunk to improve result

## I just use basic chunking strategy, maybe advanced chunking can help improve RAG QA system (late chunking, semantic chunking) or use longer context embedding model